In [1]:
import pickle
import numpy as np
import json
import scipy.sparse as sp
from tqdm import tqdm
from collections import defaultdict

In [2]:
def get_ids(split):
    ids = [i for i in pid[split] if 'gab' in i]
    return ids

In [3]:
data_path = '../../data/commons/dataset.json'
pid_path = '../../data/commons/post_id_divisions.json'

with open(pid_path) as f:
    pid = json.load(f)
with open(data_path) as f:
    data = json.load(f)

label_dict = {'normal': 0, 'offensive': 1, 'hatespeech': 2}

HatexplainGabIds = get_ids("train") + get_ids("val") + get_ids("test")
len(HatexplainGabIds)

10423

In [4]:
PostAdj = sp.load_npz("../../data/commons/A_full.npz").tocsr()
PostAdj = PostAdj.multiply(PostAdj.multiply(PostAdj))

In [5]:
GabId2PostAdjIdx = pickle.load(open("../../data/commons/id_2_idx.pkl", "rb"))

In [6]:
AdjList = dict()
for HatexplainGabId in tqdm(HatexplainGabIds):
    GabId = int(HatexplainGabId[:-4])
    PostAdjIdx = GabId2PostAdjIdx[GabId]
    cx = PostAdj[PostAdjIdx].tocoo()
    NeighborsIdxs = [idx for idx in cx.col]
    AdjList[GabId] = NeighborsIdxs

100%|██████████| 10423/10423 [00:10<00:00, 1001.69it/s]


In [7]:
pickle.dump(AdjList, open("neighbors_list_dict.pkl", "wb"))

In [8]:
GabData = pickle.load(open("../../../sumegh/GabData/Final_Posts.pkl", "rb"))

In [9]:
HatexplainGabUsers = set()
for HatexplainGabId in tqdm(HatexplainGabIds):
# for HatexplainGabId in HatexplainGabIds:
    GabId = int(HatexplainGabId[:-4])
    GabUser = GabData[GabId]['userid']
    HatexplainGabUsers.add(GabUser)
len(HatexplainGabUsers)

100%|██████████| 10423/10423 [00:00<00:00, 60114.56it/s]


3209

In [14]:
UserPostsIdxList = dict()
for HatexplainGabUser in sorted(list(HatexplainGabUsers)):
    UserPostsIdxList[HatexplainGabUser] = []
for idx in tqdm(GabData):
    Datum = GabData[idx]
    UserId = Datum['userid']
    if UserId in HatexplainGabUsers and Datum['post_body'] is not None:
        UserPostsIdxList[UserId].append(idx)

100%|██████████| 21207961/21207961 [01:32<00:00, 228320.13it/s]


In [11]:
HatexplainGabId2GabUserId = dict()
for HatexplainGabId in tqdm(HatexplainGabIds):
    GabId = int(HatexplainGabId[:-4])
    GabUser = GabData[GabId]['userid']
    HatexplainGabId2GabUserId[GabId] = GabUser

100%|██████████| 10423/10423 [00:00<00:00, 23259.87it/s]


In [15]:
pickle.dump(HatexplainGabId2GabUserId, open("post_id_2_user_id.pkl", "wb"))
pickle.dump(UserPostsIdxList, open("user_id_posts_idx_list.pkl", "wb"))